# Home Inventory Demo

This Demo uses the Google Cloud Vision API to create a home inventory, using casually collected photos of a home.

In [1]:
# Uncomment and run this to install dependencies.
# pip install --upgrade google-cloud-vision

In [4]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision_v1, vision_v1p4beta1

from util import helpers, data_classes

In [5]:
# Instantiates a client
client = vision_v1.ImageAnnotatorClient()

pclient = vision_v1p4beta1.ImageAnnotatorClient()

In [6]:
request = {
    'image': {
        'source': {'image_uri': 'gs://mikebernico-sandbox-sf-home-inventory-demo/MVIMG_20200310_112705.jpg'},
    },
}

In [5]:
response = client.annotate_image(request)

In [7]:
response = pclient.annotate_image(request)

In [12]:
response.product_search_results

In [6]:
home_objects = []

In [10]:
"""Example localized_object_annotation
[mid: "/m/040b_t"
name: "Refrigerator"
score: 0.9428831338882446
bounding_poly {
  normalized_vertices {
    x: 0.02349490113556385
    y: 0.3256329596042633
  }
  normalized_vertices {
    x: 0.478804349899292
    y: 0.3256329596042633
  }
  normalized_vertices {
    x: 0.478804349899292
    y: 0.9935383796691895
  }
  normalized_vertices {
    x: 0.02349490113556385
    y: 0.9935383796691895
  }
}
]
"""
home_objects = [] # list to hold home objects
width = response.full_text_annotation.pages[0].width # image width, needed to normalize text_annotations
height = response.full_text_annotation.pages[0].height # image height, needed to normalize text_annotations

# Parse text_annotations
text_annotations = []
for a in response.text_annotations:
    new_text_annotation = data_classes.text_annotation(a.description, height, width)
    new_text_annotation.update_location(a.bounding_poly.vertices[0], a.bounding_poly.vertices[3])
    text_annotations.append(new_text_annotation)
    
# Parse object responses
for r in response.localized_object_annotations:
    new_home_object = data_classes.home_object(r.name, r.score)
    new_home_object.update_location(r.bounding_poly.normalized_vertices[0], r.bounding_poly.normalized_vertices[3])
    home_objects.append(new_home_object)
    
    # Append intersecting text annotations onto home objects
    for t in text_annotations:
        iou = helpers.bb_intersection_over_union(new_home_object.location, t.location)
        if iou > 0.5:
            new_home_object.object_text += " " + t.text
    


In [11]:
for h in home_objects:
    print(h.name)
    print(h.object_text)

Refrigerator
 11:26
---
BOSCH
COCOA
KISSES
Live
e BOSCH
MILAGRO
UTEN TREE
DO
0000
BOSCH

Microwave oven
 11 : 26 - - - BOSCH DO 0000
Oven
 Live MILAGRO UTEN TREE DO 0000 BOSCH


In [8]:
for h in home_objects:
    print(h.name)
    print("**--------**")
    for t in text_annotations:
        iou = helpers.bb_intersection_over_union(h.location, t.location)
        if iou > 0.5:
            print (t.text)

Refrigerator
**--------**
11:26
---
BOSCH
COCOA
KISSES
Live
e BOSCH
MILAGRO
UTEN TREE
DO
0000
BOSCH

Microwave oven
**--------**
11
:
26
-
-
-
BOSCH
DO
0000
Oven
**--------**
Live
MILAGRO
UTEN
TREE
DO
0000
BOSCH
